In [4]:
from rich.traceback import install; install()
%load_ext autoreload
%autoreload 2

import sys; sys.path.append(2*"../"); sys.path.append("../../GLOP")

import torch

from rl4co.envs import TSPEnv, CVRPEnv, SHPPEnv
from rl4co.models.zoo import (
    AttentionModel,
    AttentionModelPolicy,
    GLOPOriPolicy,
    GLOPOri,
)
from rl4co.utils.trainer import RL4COTrainer
from rl4co.utils.ops import batchify, gather_by_index

from GLOP.utils import load_model

device = "cuda:0"

In [2]:
# SECTION: Follow GLOP to create the opts
class Config:
    def __init__(self):
        self.problem_size = 100
        self.revision_lens = [10, 20]
        self.revision_iters = [5, 5]
        self.decode_strategy = 'greedy'
        self.width = 10
        self.no_aug = True
        self.seed = 1
        self.val_size = 100
        self.n_epochs = 10
        self.steps_per_epoch = 256
        self.checkpoint_path = ""
        self.max_norm = 1.0
        self.units = 48
        self.no_clip = False
        self.batch_size = 16
        self.depth = 12
        self.no_prune = False
        self.problem_type = "tsp"
        self.revisers = []

# Example of creating an instance of Config
opts = Config()

# Load GLOP revisers from checkpoints
revisers = []
for reviser_size in opts.revision_lens:
    reviser_path = f'../../GLOP/pretrained/Reviser-stage2/reviser_{reviser_size}/epoch-299.pt'
    reviser, _ = load_model(reviser_path, is_local=True)
    revisers.append(reviser)
for reviser in revisers:
    reviser.to(device)
    reviser.eval()
    reviser.set_decode_type(opts.decode_strategy)    
opts.revisers = revisers

  [*] Loading model from ../../GLOP/pretrained/Reviser-stage2/reviser_10/epoch-299.pt
  [*] Loading model from ../../GLOP/pretrained/Reviser-stage2/reviser_20/epoch-299.pt


In [6]:
# SECTION: GLOPOri Policy Checking
# Init GLOPOri Policy
batch_size = 3
n_samples = 10

env = CVRPEnv(generator_params={"num_loc": 50})

policy = GLOPOriPolicy(
    env_name=env.name,
    n_samples=n_samples,
    opts=opts
)
policy = policy.to(device)
td = env.reset(batch_size=[3]).to(device)

out = policy(
    td=td.clone(),
    env=env,
    phase="test",
    return_actions=True,
)

In [7]:
# SECTION: GLOP Model Checking
model = GLOPOri(
    env=env,
    batch_size=16,
    opts=opts,
    train_data_size=1000,
    val_data_size=1000,
    test_data_size=1000,
    n_samples=n_samples,
)
model = model.to(device)

/home/cbhua/miniconda/envs/rl4co-private/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'env' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['env'])`.
/home/cbhua/miniconda/envs/rl4co-private/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.


In [8]:
trainer = RL4COTrainer(
    max_epochs=100,
    accelerator="gpu",
    devices=1,
    logger=None,
)
trainer.fit(model)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/cbhua/miniconda/envs/rl4co-private/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/home/cbhua/miniconda/envs/rl4co-private/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'policy' removed from hparams because it cannot be pickled
val_file not set. Generating dataset instead
test_file not set. Gene

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/cbhua/miniconda/envs/rl4co-private/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


/home/cbhua/miniconda/envs/rl4co-private/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Epoch 0:   8%|▊         | 5/63 [00:10<02:04,  0.46it/s, v_num=1, train/reward=19.70, train/loss=0.575]

/home/cbhua/miniconda/envs/rl4co-private/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
